In [1]:
import os
import pandas as pd
import math
import numpy as np
from tqdm import tqdm, trange
from seqeval.metrics import classification_report,accuracy_score,f1_score

import torch
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig, BertForTokenClassification, \
                            get_linear_schedule_with_warmup

from sklearn import metrics
from ner_config import *
from ner_utils import *
from data_processor import i2b2Dataset, pad_batch
from model import Bert_BiLSTM_CRF

from scipy.special import softmax


import stanza
os.chdir('..')

In [2]:
try:
    nlp = stanza.Pipeline(lang="en", processors="tokenize")
except Exception:
    stanza.download("en")
    nlp = stanza.Pipeline(lang="en", processors="tokenize")


2022-08-04 03:21:53 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2022-08-04 03:21:53 INFO: Use device: gpu
2022-08-04 03:21:53 INFO: Loading: tokenize
2022-08-04 03:21:57 INFO: Done loading processors!


In [10]:
long_text = '''
Two days prior to admission, she was started on a prednisone taper and one day prior to admission she required oxygen at home in order to maintain oxygen saturation greater than 90%.  She has also been on levofloxacin and nebulizers, and was not getting better, and presented to the [**Hospital1 18**] Emergency Room.  In the [**Hospital3 **] Emergency Room, her oxygen saturation was 100% on CPAP.  She was not able to be weaned off of this despite nebulizer treatment and Solu-Medrol 125 mg IV x2.  Review of systems is negative for the following:  Fevers, chills, nausea, vomiting, night sweats, change in weight, gastrointestinal complaints, neurologic changes, rashes, palpitations, orthopnea.  Is positive for the following: Chest pressure occasionally with shortness of breath with exertion, some shortness of breath that is positionally related, but is improved with nebulizer treatment.  PAST MEDICAL HISTORY: 1. COPD.  Last pulmonary function tests in [**2117-11-3**] demonstrated a FVC of 52% of predicted, a FEV1 of 54% of predicted, a MMF of 23% of predicted, and a FEV1:FVC ratio of 67% of predicted, that does not improve with bronchodilator treatment.  The FVC, however, does significantly improve with bronchodilator treatment consistent with her known reversible air flow obstruction in addition to an underlying restrictive ventilatory defect.  The patient has never been on home oxygen prior to this recent episode.  She has never been on steroid taper or been intubated in the past. 2. Lacunar CVA.  MRI of the head in [**2114-11-4**] demonstrates "mild degree of multiple small foci of high T2 signal within the white matter of both cerebral hemispheres as well as the pons, in the latter region predominantly to the right of midline.  The abnormalities, while nonspecific in etiology, are most likely secondary to chronic microvascular infarction.  There is no mass, lesion, shift of the normal midline strictures or hydrocephalus.  The major vascular flow patterns are preserved. 
'''

In [5]:
DEVICE = torch.device('cuda:1')
ckpt_name = 'best_NER_bert_crf'

In [6]:
model = Bert_BiLSTM_CRF(tag2idx).to(DEVICE)
tokenizer = BertTokenizer.from_pretrained(BASE_MODEL)
ckpt = torch.load(f"{NER_MODEL_SAVED_DIR}/{ckpt_name}.ckpt", map_location=DEVICE)
model.load_state_dict(ckpt['model'])

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [11]:
# input a whole sentence as a list
def create_query(sentence, tokenizer):

    temp_token = ['[CLS]']
    # word_list = sentence.split()
    for word in sentence:
        temp_token.extend(tokenizer.tokenize(word))
    temp_token = temp_token[:MAX_LEN - 1]
    temp_token.append('[SEP]')
    input_id = tokenizer.convert_tokens_to_ids(temp_token)
    padding_len = MAX_LEN - len(input_id)
    input_id = input_id + ([0] * padding_len)
    tokenized_texts = torch.LongTensor([input_id])
    # print(input_id)
    # attention_masks = [[int(i>0) for i in input_id]]
    attention_masks = (tokenized_texts > 0)

    return temp_token, tokenized_texts, attention_masks


In [12]:
all_sentences = []
all_tags = []
doc = nlp(long_text)
for i, sentence in enumerate(doc.sentences):
    word_list = [token.text for token in sentence.tokens]
    # print(word_list)
    temp_token, sentence_tensor, mask = create_query(word_list, tokenizer)
    sentence_tensor = sentence_tensor.to(DEVICE)
    mask = mask.to(DEVICE)
    y = model(sentence_tensor, None, mask, is_test=True)
    y = y[0]
    y_tag = [idx2tag[i] for i in y]
    pretok_sent = ''
    pretags = ''
    for i in range(1, len(temp_token)-1):
        if temp_token[i].startswith("##"):
            pretok_sent += temp_token[i][2:]
        else:
            pretok_sent += f" {temp_token[i]}"
            pretags += f" {y[i]}"
    pretok_sent = pretok_sent[1:]
    pretags = pretags[1:]
    s = pretok_sent.split()
    t = pretags.split()
    all_sentences.append(s)
    all_tags.append([idx2tag.get(int(x)) for x in t])
# print(all_sentences, all_tags)
# print(all_sentences)

In [39]:
import copy
def entity_extractor(all_sentences, all_tags):
    sentences_with_problem = []
    all_problems_in_text_tmp = []
    all_treatment_in_text = []
    all_test_in_text = []

    for s, t in zip(all_sentences, all_tags):
        flag_treatment, flag_problem, flag_test = 0, 0, 0
        problem_in_sentence = ''
        treatment_in_sentence = []
        test_in_sentence = []
        for i in range(len(t)):
            if t[i] == 'B-problem' or t[i] == 'I-problem' and flag_problem == 0: 
                flag_problem = 1
                # if there is an entity, add the index of sentence to a list
                # sentences_with_problem.append(n)
                # append the index of entity to a list
                if problem_in_sentence:
                    problem_in_sentence = f'{problem_in_sentence}| {str(i)}'
                else:
                    problem_in_sentence += str(i)
            elif t[i] == 'I-problem' or t[i] == 'X' and flag_problem == 1:
                problem_in_sentence = f'{problem_in_sentence} {str(i)}'
                
                
            elif t[i] == 'B-test' or t[i] == 'I-test':
                flag_test = 1
                test_in_sentence.append(i)
            elif t[i] == 'X' and flag_test == 1:
                test_in_sentence.append(i)
            elif t[i] == 'B-treatment' or t[i] == 'I-treatment':
                flag_treatment = 1
                treatment_in_sentence.append(i)
            elif t[i] == 'X' and flag_treatment == 1:
                treatment_in_sentence.append(i)
            elif t[i] in ['O', 'X']:
                flag_treatment, flag_problem, flag_test = 0, 0, 0
                # print(s[i], end=' ')
        print(problem_in_sentence)
        all_problems_in_text_tmp.append(problem_in_sentence)
        all_treatment_in_text.append(treatment_in_sentence)
        all_test_in_text.append(test_in_sentence)
            
    # create sentences with '[entity]' tag
    all_problems_in_text = []
    sentences_with_problem = []
    for sentence, problem_index in zip(all_sentences, all_problems_in_text_tmp):
        # print(problem_index)
        if problem_index:
            index = problem_index.split('|')
            tmp = [i.split() for i in index]
            all_problems_in_text.append(tmp)
            for i_list in tmp:
                s = copy.deepcopy(sentence)
                s.insert(int(i_list[-1])+1, '[entity]')
                s.insert(int(i_list[0]), '[entity]')
                s = ' '.join(s)
                sentences_with_problem.append(s)
        else:
            # sentences_with_problem.append(sentence)
            all_problems_in_text.append(problem_index)

    return sentences_with_problem, all_problems_in_text, all_treatment_in_text, all_test_in_text


In [40]:
sentences_with_problem, all_problems_in_text, all_treatment_in_text, all_test_in_text = entity_extractor(all_sentences, all_tags)







9| 11| 13| 15| 17 18| 20 21 22| 24 25| 27 28| 30| 32| 34
6 7| 10 11 12| 14| 16 17 18 19| 22 23
6

0 1| 13 14 15 16 17 18| 22 23 24 25 26
10 11 12

2 3
18 19| 21 22 23| 25 26 27| 29 30 31
1| 13 14 15
3| 5| 7| 9 10 11 12| 14
2 3


In [38]:
all_test_in_text

[[28, 29],
 [],
 [],
 [],
 [3, 4, 5, 10],
 [],
 [],
 [],
 [],
 [0, 1, 2, 3, 17, 18, 25, 26, 33, 34, 42, 43, 45, 46],
 [],
 [],
 [],
 [],
 [0, 1, 2, 3],
 [],
 [],
 []]

In [25]:
sentences_with_problem

['review of systems is negative for the following : [entity] fevers [entity] , chills , nausea , vomiting , night sweats , change in weight , gastrointestinal complaints , neurologic changes , rashes , palpitations , orthopnea .',
 'review of systems is negative for the following : fevers , [entity] chills [entity] , nausea , vomiting , night sweats , change in weight , gastrointestinal complaints , neurologic changes , rashes , palpitations , orthopnea .',
 'review of systems is negative for the following : fevers , chills , [entity] nausea [entity] , vomiting , night sweats , change in weight , gastrointestinal complaints , neurologic changes , rashes , palpitations , orthopnea .',
 'review of systems is negative for the following : fevers , chills , nausea , [entity] vomiting [entity] , night sweats , change in weight , gastrointestinal complaints , neurologic changes , rashes , palpitations , orthopnea .',
 'review of systems is negative for the following : fevers , chills , nausea

In [ ]:
b_tr = 'B-treatment'
i_tr = 'I-treatment'
b_t = 'B-test'
i_t = 'I-test'
b_p = 'B-problem'
i_p = 'I-problem'

flag_treatment, flag_problem, flag_test = 0, 0, 0 
for i in range(len(tags)):
    
    if tags[i] == b_tr:
        flag_treatment = 1
        print(b_tr, end=' ')
    elif tags[i] == i_tr:
        if flag_treatment == 0:
       print(b_t, end=' ')
            flag_test = 1
        else:
            print(i_t, end=' ')
    elif tags[i] == 'X' and flag_test == 1:
        print(i_t, end=' ')
        
    elif tags[i] == b_p:
        flag_problem = 1
        print(b_p, end=' ')
    elif tags[i] == i_p:
        if flag_problem == 0:
            print(b_p, end=' ')
            flag_problem = 1
        else:
            print(i_p, end=' ')   
    elif tags[i] == 'X' and flag_problem == 1 :
        print(i_p, end=' ')    
        
    elif tags[i] == 'O' or tags[i] == 'X':
        flag_treatment, flag_problem, flag_test = 0, 0, 0 
        print('O', end=' ')      print(b_tr, end=' ')
            flag_treatment = 1
        else:
            print(i_tr, end=' ')
    elif tags[i] == 'X' and flag_treatment == 1 :
            print(i_tr, end=' ')
              
    elif tags[i] == b_t:
        flag_test = 1
        print(b_t, end=' ')
    elif tags[i] == i_t:
        if flag_test == 0:
           
    

In [35]:
tags = ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-problem',
  'I-problem',
  'O',
  'O',
  'B-problem',
  'X',
  'I-problem',
  'O',
  'B-problem',
  'O',
  'O',
  'I-problem',
  'I-problem',
  'X',
  'O',
  'O',
  'B-problem',
  'I-problem',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-treatment',
  'I-treatment',
  'O']

In [34]:
b_tr = 'B-treatment'
i_tr = 'I-treatment'
b_t = 'B-test'
i_t = 'I-test'
b_p = 'B-problem'
i_p = 'I-problem'

flag_treatment, flag_problem, flag_test = 0, 0, 0 
for i in range(len(tags)):
    if tags[i] == b_tr or tags[i] == i_tr and flag_treatment == 0:
        flag_treatment = 1
        print(b_tr, end=' ')
    elif tags[i] == i_tr or tags[i] == 'X' and flag_treatment == 1:
        print(i_tr, end=' ')
        
    elif tags[i] == b_p or tags[i] == i_p and flag_problem == 0:
        flag_problem = 1
        print(b_p, end=' ')
    elif tags[i] == i_p or tags[i] == 'X' and flag_problem == 1 :
        print(i_p, end=' ')    
        
    elif tags[i] == b_t or tags[i] == i_t and flag_test == 0:
        flag_test = 1
        print(b_t, end=' ')   

    elif tags[i] == i_t or tags[i] == 'X' and flag_treatment == 1:
        print(i_t, end=' ')
            
    elif tags[i] == 'O' or tags[i] == 'X':
        flag_treatment, flag_problem, flag_test = 0, 0, 0 
        print('O', end=' ') 
    

O O O O O O B-problem I-problem O O B-problem I-problem I-problem O B-problem O O B-problem I-problem I-problem O O B-problem I-problem O O O O O B-treatment I-treatment O 

In [10]:
['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-problem',
  'I-problem',
  'O',
  'O',
  'B-problem',
  'X',
  'I-problem',
  'O',
  'B-problem',
  'O',
  'O',
  'I-problem',
  'I-problem',
  'X',
  'O',
  'O',
  'B-problem',
  'I-problem',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-treatment',
  'I-treatment',
  'O']

['B-test',
 'I-test',
 'I-test',
 'I-test',
 'O',
 'O',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'O',
 'O',
 'O',
 'I-problem',
 'O',
 'B-problem',
 'I-problem',
 'I-problem',
 'X',
 'B-problem',
 'I-problem',
 'I-problem',
 'O',
 'I-problem',
 'I-problem',
 'I-problem',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [24]:
location = {}
flag_treatment, flag_problem, flag_test = 0, 0, 0 
for i in range(1, len(t)-1):
    if t[i] == 'B-treatment':
        flag_treatment = 1
        print(print_treatment(s[i]), end=' ')
    elif t[i] == 'I-treatment' or t[i] == 'X' and flag_treatment == 1 :
        print(print_treatment(s[i]), end=' ')
    elif t[i] == 'B-test':
        flag_test = 1
        print(print_test(s[i]), end=' ')
    elif t[i] == 'I-test' or t[i] == 'X' and flag_test == 1 :
        print(print_test(s[i]), end=' ')
    elif t[i] == 'B-problem':
        flag_problem = 1
        print(print_problem(s[i]), end=' ')
    elif t[i] == 'I-problem' or t[i] == 'X' and flag_problem == 1 :
        print(print_problem(s[i]), end=' ')    
    elif t[i] == 'O':
        flag_treatment, flag_problem, flag_test = 0, 0, 0 
        print(s[i], end=' ')

[['O', 'O', 'O', 'O', 'X', 'X', 'O', 'X', 'X', 'X', 'X', 'O', 'X', 'X'], ['O', 'O', 'O', 'O', 'X', 'X', 'O', 'X', 'X', 'X', 'X', 'O', 'X', 'X'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'X', 'O', 'O', 'O', 'O', 'O', 'O', 'X', 'X', 'O', 'O', 'O', 'O', 'X', 'X'], ['O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'X', 'X', 'X', 'X', 'O', 'O', 'O', 'O', 'O', 'B-problem', 'O', 'O', 'O', 'B-treatment', 'I-treatment', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-problem', 'I-problem', 'I-problem', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-problem', 'B-problem', 'I-problem', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-treatment', 'I-treatment', 'I-treatment', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-treatment', 'O', 'O', 'O', 'O', 'O', 'O', 'B-test', 'I-test', 'O', 'O', 'O', 'X', 'O'], ['O', 'O', 'O', 'O', 'O', 'B-treatment', 'O', 'B-treatment', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'X', 'X', 'O', 'O', 'X', 'X', 'O'], ['O', 'O', 'O'], ['O', 'O', 'O', 'X', '

In [32]:
l = ['O', 'O', 'O', 'O', 'B-problem', 'I-problem', 'O', 'O', 'O', 'O', 'I-problem', 'O', 'O', 'O', 'O', 'O', 'B-problem', 'I-problem', 'I-problem', 'I-problem', 'I-problem', 'O']

In [35]:
len(l)

22

In [33]:
i = all_tags.index(l)

In [37]:
all_sentences[i]

['it',
 'was',
 'felt',
 'that',
 'the',
 'leukocytosis',
 'was',
 'secondary',
 'to',
 'both',
 'steroids',
 'as',
 'well',
 'as',
 'question',
 'of',
 'a',
 'left',
 'lower',
 'lobe',
 'pneumonia',
 '.']

In [ ]:
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-problem', 'I-problem', 'O', 'I-problem', 'I-problem', 'I-problem', 'O', 'O', 'O', 'O', 'O', 'O']

In [9]:
s = '[CLS] admission date : [ * * 2118 - 6 - 2 * * ] [SEP]'

In [10]:
s[1:]

'CLS] admission date : [ * * 2118 - 6 - 2 * * ] [SEP]'

In [ ]:
all_sentences = []
all_tags = []

In [ ]:

pretok_sent = ""
pretags = ""
for i, tok in enumerate(temp_token):
    if tok.startswith("##"):
        pretok_sent += tok[2:]
    else:
        pretok_sent += f" {tok}"
        pretags += f" {result[i]}"
pretok_sent = pretok_sent[1:]
pretags = pretags[1:]
s = pretok_sent.split()
t = pretags.split()
all_sentences.append(s)
all_tags.append(t)

In [20]:
# sentence = 'The FVC, however, does significantly improve with bronchodilator treatment consistent with her known reversible air flow obstruction in addition to an underlying restrictive ventilatory defect.'
sentence = 'At 9am the morning of admission he passed a large , bloody bowel movement and came to the Michael .'

In [21]:
temp_token, sentence_tensor, mask = create_query(sentence, tokenizer)
sentence_tensor = sentence_tensor.to(DEVICE)
mask = mask.to(DEVICE)

In [22]:
y = model(sentence_tensor, None, mask, is_test=True)

In [23]:
y = y[0]
y_tag = [idx2tag[i] for i in y]

In [24]:
y_tag

['[CLS]',
 'O',
 'O',
 'X',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-problem',
 'I-problem',
 'I-problem',
 'I-problem',
 'I-problem',
 'X',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'X',
 'X',
 'O',
 '[SEP]']

In [ ]:
def model_inference(model, input_ids):
    model.to(DEVICE)
    input_ids = input_ids.to(DEVICE)
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None,
                        attention_mask=None)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0]
    # Get NER predict result
    predict_results = logits.detach().cpu().numpy()
    result_arrays_soft = softmax(predict_results[0])

    return np.argmax(result_arrays_soft, axis=-1)

In [ ]:
def predict_entities(long_text):
    all_sentences = []
    all_tags = []
    doc = nlp(long_text)
    for i, sentence in enumerate(doc.sentences):
        # temp_token: tokenized words
        # input_ids: convert temp_token to id
        word_list = [token.text for token in sentence.tokens]
        temp_token, input_ids, attention_masks = create_query(word_list, tokenizer_ner)
        result_list = model_inference(model_ner, input_ids)
        result = [tag2name[t] for t in result_list]
        pretok_sent = ""
        pretags = ""
        for i, tok in enumerate(temp_token):
            if tok.startswith("##"):
                pretok_sent += tok[2:]
            else:
                pretok_sent += f" {tok}"
                pretags += f" {result[i]}"
        pretok_sent = pretok_sent[1:]
        pretags = pretags[1:]
        s = pretok_sent.split()
        t = pretags.split()
        all_sentences.append(s)
        all_tags.append(t)
    return all_sentences, all_tags

In [4]:
ckpt_name = 'best_NER_bert_crf'

NameError: name 'torch' is not defined

In [4]:

if ckpt_name is not None:
    if os.path.exists(f"{NER_MODEL_SAVED_DIR}/{ckpt_name}.ckpt"):
        print("Loading the pre-trained checkpoint...")
        ckpt = torch.load(f"{NER_MODEL_SAVED_DIR}/{ckpt_name}.ckpt", map_location=DEVICE)
        model.load_state_dict(ckpt['model'])
        sentence = ['[CLS]'] + list(txt) + ['[SEP]']
        infer(model, tokenizer, sentence)
    else:
        print("No such file!")
else:
    print("mode type error!")


Start infer


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


yes
Loading the pre-trained checkpoint...


NameError: name 'args' is not defined

: 

In [ ]:

tag2idx = {'B-problem': 0,
           'B-test': 1,
           'B-treatment': 2,
           'I-problem': 3,
           'I-test': 4,
           'I-treatment': 5,
           'O': 6,
           'X': 7,
           '[CLS]': 8,
           '[SEP]': 9
           }

idx2tag = {tag2idx[key]: key for key in tag2idx}

In [ ]:
idx2tag